In [16]:
import numpy as np
import pandas as pd

In [17]:
df = pd.read_csv('data/train_data_for_students.tsv', sep='\t', header=None)
df.columns = ['id', 'handle', 'tweet', 'date', 'device']
print(f'{df.shape=}')
df.head()

df.shape=(2682, 5)


,id,handle,tweet,date,device
0,845974102619906048,realDonaldTrump,Democrats are smiling in D.C. that the Freedom...,2017-03-26 15:21:58,iphone
1,846166053663191040,realDonaldTrump,General Kelly is doing a great job at the bord...,2017-03-27 04:04:42,iphone
2,835814988686233601,realDonaldTrump,"The race for DNC Chairman was, of course, tota...",2017-02-26 13:33:16,android
3,835817351178301440,realDonaldTrump,For first time the failing @nytimes will take ...,2017-02-26 13:42:39,android
4,835916511944523777,realDonaldTrump,"Russia talk is FAKE NEWS put out by the Dems, ...",2017-02-26 20:16:41,android


In [18]:
df.device.value_counts()


android                                                                                1683
iphone                                                                                  755
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                      201
<a href="http://www.twitter.com" rel="nofollow">Twitter for BlackBerry</a>               13
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>       9
<a href="http://twitter.com/#!/download/ipad" rel="nofollow">Twitter for iPad</a>         4
<a href="http://instagram.com" rel="nofollow">Instagram</a>                               3
<a href="https://periscope.tv" rel="nofollow">Periscope.TV</a>                            2
<a href="http://www.facebook.com/twitter" rel="nofollow">Facebook</a>                     1
Name: device, dtype: int64

There is issue with the label column. There are values that not iphone / android. WTF?

In [19]:
df = df[(df.device == 'iphone') | (df.device == 'android')]
df.device.value_counts()

android    1683
iphone      755
Name: device, dtype: int64

In [52]:
# Create folds

from sklearn.model_selection import StratifiedKFold

num_folds = 3

skf = StratifiedKFold(n_splits=num_folds, random_state=11, shuffle=True)
df['fold'] = -1
df.reset_index(drop=True, inplace=True)

print(f'Number of folds: {num_folds}, total samples: {len(df)}')
for fold, (train_idx, val_idx) in enumerate(skf.split(X=df.drop(['device'], axis=1), y=df.device)):
  print(f'fold: {fold}, num samples: {len(val_idx)}')
  df.loc[val_idx, 'fold'] = fold

Number of folds: 3, total samples: 2438
fold: 0, num samples: 813
fold: 1, num samples: 813
fold: 2, num samples: 812


,id,handle,tweet,date,device,fold
0,845974102619906048,realDonaldTrump,Democrats are smiling in D.C. that the Freedom...,2017-03-26 15:21:58,iphone,0
1,846166053663191040,realDonaldTrump,General Kelly is doing a great job at the bord...,2017-03-27 04:04:42,iphone,2
2,835814988686233601,realDonaldTrump,"The race for DNC Chairman was, of course, tota...",2017-02-26 13:33:16,android,0
3,835817351178301440,realDonaldTrump,For first time the failing @nytimes will take ...,2017-02-26 13:42:39,android,2
4,835916511944523777,realDonaldTrump,"Russia talk is FAKE NEWS put out by the Dems, ...",2017-02-26 20:16:41,android,0
...,...,...,...,...,...,...
2433,740524891108200448,realDonaldTrump,@lovusa4: @RichardWeaving Why @foxandfriends @...,2016-06-08 15:44:26,android,0
2434,740641413335908352,realDonaldTrump,MAKE AMERICA GREAT AGAIN! #AmericaFirst #Trump...,2016-06-08 23:27:28,iphone,0
2435,831169341358014475,realDonaldTrump,Today I will meet with Canadian PM Trudeau and...,2017-02-13 17:53:08,iphone,0
2436,831185943482793986,realDonaldTrump,Welcome to the @WhiteHouse Prime Minister @Jus...,2017-02-13 18:59:06,iphone,2


In [5]:
import re
def preprocess(txt: str):
    """
    All the preprocessing of the text
    """

    # remove urls from tweet
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    txt = re.sub(url_regex, '', txt)

    return txt

In [6]:
from datasets import Dataset

temp = df[['tweet', 'device']]
temp.columns = ['tweet', 'label']
temp
dataset = Dataset.from_pandas(temp[['tweet', 'label']])
dataset[0] 

{'tweet': 'Democrats are smiling in D.C. that the Freedom Caucus, with the help of Club For Growth and Heritage, have saved Planned Parenthood &amp; Ocare!',
 'label': 'iphone',
 '__index_level_0__': 0}

In [7]:
temp


,tweet,label
0,Democrats are smiling in D.C. that the Freedom...,iphone
1,General Kelly is doing a great job at the bord...,iphone
2,"The race for DNC Chairman was, of course, tota...",android
3,For first time the failing @nytimes will take ...,android
4,"Russia talk is FAKE NEWS put out by the Dems, ...",android
...,...,...
2677,@lovusa4: @RichardWeaving Why @foxandfriends @...,android
2678,MAKE AMERICA GREAT AGAIN! #AmericaFirst #Trump...,iphone
2679,Today I will meet with Canadian PM Trudeau and...,iphone
2680,Welcome to the @WhiteHouse Prime Minister @Jus...,iphone
